In [1]:
import numpy as np
import pandas as pd
import math
katalog="Z:/Kuba/Studia/uw/licencjatMIM/kody/dane/"

## Wgranie danych

In [2]:
FiveFactorsD = pd.read_csv(katalog+"F-F_Research_Data_5_Factors_2x3_daily.csv",skiprows=3,nrows=14749)
FiveFactorsD.set_axis(['Date', 'Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'TB'], axis=1, inplace=True)
FiveFactorsD
MomentumD = pd.read_csv(katalog+"F-F_Momentum_Factor_daily.CSV",skiprows=13,nrows=25086)
MomentumD.set_axis(['Date', 'Mom'], axis=1, inplace=True)
MomentumD
BAAD = pd.read_csv(katalog+"Daily_BAA1986.csv",skiprows=0,nrows=9452)
BAAD.set_axis(['Date', 'BAAD'], axis=1, inplace=True)
BAAD.dropna(how='any', axis=0, inplace=True)
BAAD.reset_index(drop=True,inplace=True)
BAAD
ExpInflationM = pd.read_csv(katalog+"MICH_EXPINF1YR_1978.csv",skiprows=0,nrows=531)
ExpInflationM.set_axis(['DateM', 'ExpInf'], axis=1, inplace=True)
ExpInflationM['DateM'] = ((ExpInflationM['DateM']-ExpInflationM['DateM']%100)/100).astype('int64')
ExpInflationM['DEI']=ExpInflationM['ExpInf']-ExpInflationM['ExpInf'].shift(periods=1)
ExpInflationM
ProductionM = pd.read_csv(katalog+"INDPRO_1919.csv",skiprows=0,nrows=1239)
ProductionM.set_axis(['DateM', 'INDPRO'], axis=1, inplace=True)
ProductionM['DateM'] = ((ProductionM['DateM']-ProductionM['DateM']%100)/100).astype('int64')
ProductionM['DateY']=((ProductionM['DateM']-ProductionM['DateM']%100)/100).astype('int64')
ProductionM['MP']=np.log(ProductionM['INDPRO']/ProductionM['INDPRO'].shift(periods=1))
ProductionM['YP']=np.log(ProductionM['INDPRO']/ProductionM['INDPRO'].shift(periods=12))
ProductionM
LGB30YD = pd.read_csv(katalog+"BillsBonds1960_daily.csv",skiprows=6,nrows=15729,header=None)
LGB30YD.drop(LGB30YD.columns[[1,2,3,4,5,6,7,8,9,10]],inplace=True,axis=1)
LGB30YD.set_axis(['Date', 'LGB'], axis=1, inplace=True)
LGB30YD.reset_index(drop=True,inplace=True)
LGB30YD['DateM']=((LGB30YD['Date']-LGB30YD['Date']%100)/100).astype('int64')
LGB30YD.fillna(method='ffill',inplace=True)
LGB30YD
InflationM = pd.read_csv(katalog+"Inflation_1956.csv",skiprows=0,nrows=795)
InflationM['DateM'] = InflationM['TIME'].str.slice(0,4).astype(str)+InflationM['TIME'].str.slice(5,7).astype(str)
InflationM['DateM']=InflationM['DateM'].astype('int64')
InflationM.drop(columns=['LOCATION','INDICATOR','SUBJECT', 'MEASURE', 'FREQUENCY','Flag Codes','TIME'],inplace=True)
InflationM.set_axis(['Inf','DateM'], axis=1, inplace=True)
InflationM

Portfolios=pd.read_csv(katalog+"6_Portfolios_2x3_daily.csv",skiprows=18,nrows=25187)
Portfolios.set_axis(['Date', 'SMALL LoBM', 'ME1 BM2', 'SMALL HiBM', 'BIG LoBM',
       'ME2 BM2', 'BIG HiBM'], axis=1, inplace=True)
Portfolios

,Date,SMALL LoBM,ME1 BM2,SMALL HiBM,BIG LoBM,ME2 BM2,BIG HiBM
0,19260701,0.10,-0.09,-0.37,0.16,0.11,0.07
1,19260702,0.14,-0.19,0.31,0.52,0.47,0.30
2,19260706,0.42,0.42,0.19,0.30,0.10,-0.24
3,19260707,0.12,-0.32,-0.59,0.07,0.14,0.78
4,19260708,-0.07,-0.02,0.08,0.37,0.18,0.57
...,...,...,...,...,...,...,...
25182,20220222,-1.33,-1.38,-1.46,-1.16,-0.71,-0.78
25183,20220223,-2.33,-1.86,-1.04,-2.25,-1.37,-1.05
25184,20220224,4.14,1.88,0.41,2.74,0.47,-1.23
25185,20220225,1.95,2.21,2.79,1.93,2.73,3.50


In [37]:
LGB30YD.iloc[6274]

Date     19860117.0
LGB             9.4
DateM      198601.0
Name: 6273, dtype: float64

## Obróbka i łączenie danych

In [3]:

dane1 = FiveFactorsD.merge(MomentumD, how='inner', on='Date')
dane1
(FiveFactorsD.shape, MomentumD.shape, dane1.shape)
dane1['DateM']=((dane1['Date']-dane1['Date']%100)/100).astype('int64')

dane2= pd.merge(dane1,ProductionM,how='inner',on='DateM')
dane2
(dane1.shape,ProductionM.shape ,dane2.shape)


dane3=pd.merge(dane2,ExpInflationM,how='inner',on='DateM')
(dane2.shape,ExpInflationM.shape ,dane3.shape)

dane4=pd.merge(dane3,InflationM,how='inner',on='DateM')
(dane3.shape,ExpInflationM.shape ,dane4.shape)
# dane4

dane6=pd.merge(dane4,BAAD,how='inner',on='Date')
(dane4.shape,BAAD.shape ,dane6.shape)
dane6

#TUTAJ TRACONE SA OBSERWACJE
dane7=pd.merge(dane6,LGB30YD,how='left',on='Date')
(dane6.shape,LGB30YD.shape ,dane7.shape)
dane7
dane7.to_excel('dane7.xlsx')
dane7

,Date,Mkt-RF,SMB,HML,RMW,CMA,TB,Mom,DateM_x,INDPRO,DateY,MP,YP,ExpInf,DEI,Inf,BAAD,LGB,DateM_y
0,19860102,-0.63,0.88,0.39,-0.41,0.33,0.025,-0.72,198601,55.8793,1986,0.005086,0.020959,2.9,-0.6,3.886256,11.38,9.28,198601
1,19860103,0.56,-0.08,0.21,-0.14,0.30,0.025,0.01,198601,55.8793,1986,0.005086,0.020959,2.9,-0.6,3.886256,11.35,9.30,198601
2,19860106,-0.04,0.04,0.10,-0.22,-0.04,0.025,0.00,198601,55.8793,1986,0.005086,0.020959,2.9,-0.6,3.886256,11.36,9.31,198601
3,19860107,1.38,-0.48,0.06,-0.29,-0.02,0.025,-0.03,198601,55.8793,1986,0.005086,0.020959,2.9,-0.6,3.886256,11.29,9.18,198601
4,19860108,-2.16,1.42,0.32,-0.09,0.14,0.025,0.15,198601,55.8793,1986,0.005086,0.020959,2.9,-0.6,3.886256,11.34,9.36,198601
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9043,20220125,-1.43,-0.17,2.79,-0.79,1.62,0.000,0.27,202201,103.0309,2022,0.013800,0.035800,4.9,0.1,7.479873,3.69,2.12,202201
9044,20220126,-0.30,-1.06,0.09,0.44,-0.06,0.000,1.40,202201,103.0309,2022,0.013800,0.035800,4.9,0.1,7.479873,3.69,2.16,202201
9045,20220127,-0.78,-1.45,0.77,1.04,0.97,0.000,0.29,202201,103.0309,2022,0.013800,0.035800,4.9,0.1,7.479873,3.67,2.09,202201
9046,20220128,2.45,-0.63,-2.07,0.16,0.24,0.000,-0.29,202201,103.0309,2022,0.013800,0.035800,4.9,0.1,7.479873,3.69,2.07,202201


In [4]:
dane7['UI']=dane7['Inf']-dane7['ExpInf'].shift(periods=1)
dane7['RHO']=dane7['TB'].shift(periods=1)-dane7['Inf']
dane7['URP']=dane7['BAAD']-dane7['LGB']
dane7['UTS']=dane7['LGB']-dane7['TB'].shift(periods=1)
# dane7.drop(['DateM_x','DateM_y','DateY','INDPRO','TB','Inf','LGB','BAAD'], axis=1,inplace=True)

In [5]:
dane7.describe()
dane7

,Date,Mkt-RF,SMB,HML,RMW,CMA,TB,Mom,DateM_x,INDPRO,...,ExpInf,DEI,Inf,BAAD,LGB,DateM_y,UI,RHO,URP,UTS
0,19860102,-0.63,0.88,0.39,-0.41,0.33,0.025,-0.72,198601,55.8793,...,2.9,-0.6,3.886256,11.38,9.28,198601,NaN,NaN,2.10,NaN
1,19860103,0.56,-0.08,0.21,-0.14,0.30,0.025,0.01,198601,55.8793,...,2.9,-0.6,3.886256,11.35,9.30,198601,0.986256,-3.861256,2.05,9.275
2,19860106,-0.04,0.04,0.10,-0.22,-0.04,0.025,0.00,198601,55.8793,...,2.9,-0.6,3.886256,11.36,9.31,198601,0.986256,-3.861256,2.05,9.285
3,19860107,1.38,-0.48,0.06,-0.29,-0.02,0.025,-0.03,198601,55.8793,...,2.9,-0.6,3.886256,11.29,9.18,198601,0.986256,-3.861256,2.11,9.155
4,19860108,-2.16,1.42,0.32,-0.09,0.14,0.025,0.15,198601,55.8793,...,2.9,-0.6,3.886256,11.34,9.36,198601,0.986256,-3.861256,1.98,9.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9043,20220125,-1.43,-0.17,2.79,-0.79,1.62,0.000,0.27,202201,103.0309,...,4.9,0.1,7.479873,3.69,2.12,202201,2.579873,-7.479873,1.57,2.120
9044,20220126,-0.30,-1.06,0.09,0.44,-0.06,0.000,1.40,202201,103.0309,...,4.9,0.1,7.479873,3.69,2.16,202201,2.579873,-7.479873,1.53,2.160
9045,20220127,-0.78,-1.45,0.77,1.04,0.97,0.000,0.29,202201,103.0309,...,4.9,0.1,7.479873,3.67,2.09,202201,2.579873,-7.479873,1.58,2.090
9046,20220128,2.45,-0.63,-2.07,0.16,0.24,0.000,-0.29,202201,103.0309,...,4.9,0.1,7.479873,3.69,2.07,202201,2.579873,-7.479873,1.62,2.070


In [59]:
# Pierwszy zestaw danych - Small.LoBM 
dane = pd.merge(Portfolios[['Date','SMALL LoBM']],dane7,how='inner', on='Date')
dane.drop('Date', axis=1,inplace=True)
dane
dane.to_excel('dane.xlsx')
